# Step-1: Import the modules

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
#SparkSession creation
spark = SparkSession.builder.appName("MovieLensRating").getOrCreate()

# Loading & Cleaning the data

In [3]:
ratings = (
    spark.read.csv(
        path="movielens/ratings.csv",
        sep= ",",
        quote='"',
        header=True,
        schema="userId INT, movieID INT, rating DOUBLE, timestamp INT",
     )
    #converting timestamp format to regular one from unixtime
     .withColumn("timestamp",f.to_timestamp(f.from_unixtime("timestamp")))
)

movies = spark.read.csv(
             path="movielens/movies.csv",
             sep=",",
             quote='"',
             header=True,
             schema="movieID INT, title STRING, genres STRING"
        )
movie_genres = (
    movies
    .withColumn("genres_array",f.split("genres","\|")) #Splitting elements by pipe symbol "|"
    .withColumn("genre", f.explode("genres_array")) #exploding array
    .select("movieID","title","genre")     #Selectig columns to store
)

#Available genre 
available_genre = movie_genres.select("genre").distinct()
#Movie with no genre/ where genre=="no genres listed"
movies_with_no_genre = movie_genres.where(f.col("genre")=="(no genres listed)")

links = spark.read.csv(
             path="movielens/links.csv",
             sep=",",
             quote='"',
             header=True,
             schema="movieId INT, imdbId STRING, tmdbId INT"
        )

tags =  spark.read.csv(
             path="movielens/tags.csv",
             sep=",",
             quote='"',
             header=True,
             schema="userId INT, movieId INT, tag STRING, timestamp INT"
        ).withColumn("timestamp",f.to_timestamp(f.from_unixtime("timestamp")))


In [4]:
#cOUNT MOVIES PER GENRE
movies_per_genre = movie_genres.groupBy("genre").count()
# movies_per_genre.show()

In [5]:
tags.show()

+------+-------+-----------------+-------------------+
|userId|movieId|              tag|          timestamp|
+------+-------+-----------------+-------------------+
|     2|  60756|            funny|2015-10-25 00:29:54|
|     2|  60756|  Highly quotable|2015-10-25 00:29:56|
|     2|  60756|     will ferrell|2015-10-25 00:29:52|
|     2|  89774|     Boxing story|2015-10-25 00:33:27|
|     2|  89774|              MMA|2015-10-25 00:33:20|
|     2|  89774|        Tom Hardy|2015-10-25 00:33:25|
|     2| 106782|            drugs|2015-10-25 00:30:54|
|     2| 106782|Leonardo DiCaprio|2015-10-25 00:30:51|
|     2| 106782|  Martin Scorsese|2015-10-25 00:30:56|
|     7|  48516|     way too long|2007-01-25 06:08:45|
|    18|    431|        Al Pacino|2016-05-02 02:39:25|
|    18|    431|         gangster|2016-05-02 02:39:09|
|    18|    431|            mafia|2016-05-02 02:39:15|
|    18|   1221|        Al Pacino|2016-04-27 00:35:06|
|    18|   1221|            Mafia|2016-04-27 00:35:03|
|    18|  

In [6]:
#Joining movies table with tags on unique id "movieId": left join
opinions = (
          movies
          .join(tags,["movieId"],"left")
          .withColumnRenamed("timestamp","timestamp_tag")
          .select("userId","movieID","title","tag","timestamp_tag")
    )

In [7]:
opinions.show(5,False)

+------+-------+----------------+--------------+-------------------+
|userId|movieID|title           |tag           |timestamp_tag      |
+------+-------+----------------+--------------+-------------------+
|567   |1      |Toy Story (1995)|fun           |2018-05-02 23:33:33|
|474   |1      |Toy Story (1995)|pixar         |2006-01-14 07:47:05|
|336   |1      |Toy Story (1995)|pixar         |2006-02-04 14:36:04|
|474   |2      |Jumanji (1995)  |game          |2006-01-16 06:39:12|
|62    |2      |Jumanji (1995)  |Robin Williams|2018-06-13 03:51:47|
+------+-------+----------------+--------------+-------------------+
only showing top 5 rows



In [8]:
#Joining opinion table with rating on two unique ids i.e "movieId" and "userId": inner join
opinions_ext = (
            opinions
            .join(ratings,["movieID","userId"],"inner")
            .select("userId","movieID","title","tag","rating","timestamp_tag","timestamp")
        )
opinions_ext.show(10,False)

+------+-------+----------------------------------+----------------+------+-------------------+-------------------+
|userId|movieID|title                             |tag             |rating|timestamp_tag      |timestamp          |
+------+-------+----------------------------------+----------------+------+-------------------+-------------------+
|567   |1      |Toy Story (1995)                  |fun             |3.5   |2018-05-02 23:33:33|2018-05-02 23:33:21|
|474   |1      |Toy Story (1995)                  |pixar           |4.0   |2006-01-14 07:47:05|2001-01-04 07:36:00|
|336   |1      |Toy Story (1995)                  |pixar           |4.0   |2006-02-04 14:36:04|2005-07-24 22:48:49|
|474   |2      |Jumanji (1995)                    |game            |3.0   |2006-01-16 06:39:12|2003-03-05 22:53:34|
|62    |2      |Jumanji (1995)                    |Robin Williams  |4.0   |2018-06-13 03:51:47|2018-06-13 03:51:30|
|62    |2      |Jumanji (1995)                    |magic board game|4.0 

# Aggregating 

In [14]:
#Aggregating on the basis  of movieId
ratings.groupBy("movieID").agg(
    f.count("*").alias("Total No. of Rating"),
    f.min("rating").alias("Min Rating"),
    f.max("rating").alias("Max Rating"),
    f.avg("rating").alias("Avg Rating"),
    f.min("timestamp").alias("Earliest"),
    f.max("timestamp").alias("Latest"),
).show(10,False)

+-------+-------------------+----------+----------+------------------+-------------------+-------------------+
|movieID|Total No. of Rating|Min Rating|Max Rating|Avg Rating        |Earliest           |Latest             |
+-------+-------------------+----------+----------+------------------+-------------------+-------------------+
|1580   |165                |0.5       |5.0       |3.487878787878788 |1997-07-07 17:07:18|2018-07-22 18:30:52|
|2366   |25                 |1.5       |5.0       |3.64              |1999-11-04 20:23:49|2018-02-20 15:20:35|
|3175   |75                 |1.0       |5.0       |3.58              |1999-12-26 19:01:31|2018-06-25 10:07:19|
|1088   |42                 |1.0       |5.0       |3.369047619047619 |1997-04-07 12:36:08|2018-01-17 06:52:47|
|32460  |4                  |3.5       |5.0       |4.25              |2011-12-19 00:21:21|2017-04-22 01:12:30|
|44022  |23                 |1.0       |4.5       |3.217391304347826 |2006-10-25 23:02:59|2018-03-07 12:38:56|
|

In [25]:
tags.groupby("movieID").agg(
    f.collect_set("tag").alias("Tags"),
    f.count("tag").alias("Total Tags"),
    f.collect_set("userId").alias("User Ids"),
    f.count("userId").alias("UID count"),
    f.min("timestamp").alias("First_tagged_date"),
    f.max("timestamp").alias("Last_tagged_date"),

).show(10,False)

+-------+-----------------------+----------+--------+---------+-------------------+-------------------+
|movieID|Tags                   |Total Tags|User Ids|UID count|First_tagged_date  |Last_tagged_date   |
+-------+-----------------------+----------+--------+---------+-------------------+-------------------+
|471    |[hula hoop]            |1         |[474]   |1        |2006-01-16 06:39:07|2006-01-16 06:39:07|
|1088   |[music, dance]         |2         |[474]   |2        |2006-01-27 01:20:56|2006-01-27 01:20:56|
|1580   |[aliens]               |1         |[474]   |1        |2006-01-14 07:25:19|2006-01-14 07:25:19|
|1645   |[lawyers]              |1         |[474]   |1        |2006-01-16 06:14:55|2006-01-16 06:14:55|
|1959   |[adultery, Africa]     |2         |[474]   |2        |2006-01-23 20:58:43|2006-01-23 20:58:43|
|2122   |[Stephen King]         |1         |[474]   |1        |2006-01-16 06:08:16|2006-01-16 06:08:16|
|3175   |[spoof]                |1         |[474]   |1        |2

In [35]:
ratings.groupby("userId").agg(
    f.collect_set("movieID").alias("MOvieIDS"),
    f.count("*").alias("Ratings Given"),
    f.avg("rating").alias("Avg Rating Given"),
    f.min("rating").alias("Min Rating given"),
    f.max("rating").alias("Max Rating Given"),
).sort("Ratings Given",ascending=False).show(5)

+------+--------------------+-------------+------------------+----------------+----------------+
|userId|            MOvieIDS|Ratings Given|  Avg Rating Given|Min Rating given|Max Rating Given|
+------+--------------------+-------------+------------------+----------------+----------------+
|   414|[2338, 356, 6296,...|         2698| 3.391957005189029|             0.5|             5.0|
|   599|[3702, 356, 2594,...|         2478|2.6420500403551253|             0.5|             5.0|
|   474|[356, 6296, 2594,...|         2108| 3.398956356736243|             0.5|             5.0|
|   448|[356, 6296, 10894...|         1864|2.8473712446351933|             0.5|             5.0|
|   274|[2338, 356, 6702,...|         1346| 3.235884101040119|             0.5|             5.0|
+------+--------------------+-------------+------------------+----------------+----------------+
only showing top 5 rows

